In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Spark Kafka Connector').enableHiveSupport().getOrCreate() 
spark.conf.set("fs.defaultFS", "hdfs://167.172.61.77:8020/")

In [3]:
kafkaDF = spark.read.format("kafka").option("kafka.bootstrap.servers","167.172.61.77:9092").option("subscribe","twitter-search").load()

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.types import * 

In [5]:
schema = StructType() \
        .add("screen_name", StringType()) \
        .add("twit", StringType()) \
        .add("create_date", LongType()) \
        .add("followers_count", IntegerType()) \
        .add("friends_count", IntegerType()) \
        .add("description", StringType()) \
        .add("favorite_count", IntegerType()) \
        .add("retweet_count", IntegerType())

In [6]:
tsDF= kafkaDF.select(from_json(col("value").cast("string"), schema).alias("data")).select("data.*")

In [7]:
table = tsDF.withColumn("date", from_unixtime("create_date","dd/MM/yyyy"))

In [8]:
tableDate = table.groupBy("date").count().withColumnRenamed("date","dateF") 

In [9]:
retSum = table.groupBy("date").sum("retweet_count").withColumnRenamed("sum(retweet_count)","sum_retweet")

In [10]:
resultDF = tableDate.join(retSum, tableDate["dateF"] == retSum["date"]).select("date","count","sum_retweet");

In [11]:
resultDF.show() 
resultDF.write.format("csv").save("/tmp/ornekveriyorum56.csv")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2144142198308171> in <module> 
 1 resultDF . show ( ) 
 ----> 2 resultDF . write . format ( "csv" ) . save ( "/tmp/ornekveriyorum56.csv" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 737 self . _jwrite . save ( ) 
 738 else : 
 --> 739 self . _jwrite . save ( path ) 
 740 
 741 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o1597.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:201)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:192)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:126)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:150)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:138)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$5.apply(SparkPlan.scala:191)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:187)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:138)
	at org.apache.spark.sql.execution.QueryExecution.toRdd$lzycompute(QueryExecution.scala:117)
	at org.apache.spark.sql.execution.QueryExecution.toRdd(QueryExecution.scala:115)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:711)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:711)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1$$anonfun$apply$1.apply(SQLExecution.scala:112)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:232)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1.apply(SQLExecution.scala:98)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:835)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:74)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:184)
	at org.apache.spark.sql.DataFrameWriter.runCommand(DataFrameWriter.scala:711)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:307)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:293)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:236)
	at sun.reflect.GeneratedMethodAccessor611.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.